# SBM Allocation for Replication

James Yu, 12 July 2023

In [1]:
using JSON

include("type_allocation_base.jl")

NUMBER_OF_TYPES = 4
NUMSINKS = 4;

The allocation is pinpointed to seed zero, 4 types with 4 sinks.

In [2]:
res = SBM.estimate_parameters(NUMBER_OF_TYPES, 500 * (NUMBER_OF_TYPES-2) + 1000; SEED=0, DEBUG=true);

Total 6849 Placements (found 6849 by sequence counting, 6849 by matrix sum)
 54.596090 seconds (6 allocations: 6.453 KiB)
TYPE 1:
  Boston University
  Columbia University
  Duke University
  Harvard University
  London School of Economics and Political Science
  Massachusetts Institute of Technology
  New York University
  Northwestern University
  Princeton University
  Stanford University
  University of California Los Angeles (UCLA)
  University of California, Berkeley
  University of Chicago
  University of Maryland
  University of Michigan
  University of Pennsylvania
  University of Wisconsin, Madison
  Yale University
Total Institutions: 18

TYPE 2:
  Arizona State University
  Bocconi University
  Boston College
  Brown University
  Carnegie Mellon University
  Cornell University
  Johns Hopkins University
  Michigan State University
  National University of Singapore
  Ohio State University
  Pennsylvania State University
  Purdue University
  Stockholm School of Economics
  

8×4 Matrix{Int32}:
 196   12   84  455
 148   63   83   84
 618   43  363  725
  66    3   33  496
  66   29   41  126
 258   34  131  359
 206   18   94  447
 523  143  508  394



Estimated Placement Counts (sorted types):


8×4 Matrix{Int32}:
 496   66   33    3
 455  196   84   12
 725  618  363   43
  84  148   83   63
 126   66   41   29
 359  258  131   34
 447  206   94   18
 394  523  508  143


Estimated Placement Rates (sorted types):


8×4 Matrix{Float64}:
 1.53086    0.101852   0.0119826   0.000761035
 0.70216    0.151235   0.0152505   0.00152207
 0.263253   0.1122     0.0155069   0.00128331
 0.021309   0.0187722  0.00247709  0.00131357
 0.333333   0.0873016  0.0127607   0.00630572
 0.193635   0.0695793  0.00831271  0.00150729
 0.215942   0.0497585  0.00534243  0.000714711
 0.0261829  0.0173777  0.0039716   0.000781062


Likelihood: -27453.463489758

Check Complete


The table:

In [3]:
SBM.nice_table(res.placements, NUMBER_OF_TYPES, NUMSINKS, ["International", "Government", "Private Sector", "Teaching Universities", "Column Totals"])

┌───────────────────────┬────────┬────────┬────────┬────────┬────────────┐
│                       │ Tier 1 │ Tier 2 │ Tier 3 │ Tier 4 │ Row Totals │
├───────────────────────┼────────┼────────┼────────┼────────┼────────────┤
│                Tier 1 │    496 │     66 │     33 │      3 │        598 │
│                Tier 2 │    455 │    196 │     84 │     12 │        747 │
│                Tier 3 │    725 │    618 │    363 │     43 │       1749 │
│                Tier 4 │     84 │    148 │     83 │     63 │        378 │
│         International │    126 │     66 │     41 │     29 │        262 │
│            Government │    359 │    258 │    131 │     34 │        782 │
│        Private Sector │    447 │    206 │     94 │     18 │        765 │
│ Teaching Universities │    394 │    523 │    508 │    143 │       1568 │
│         Column Totals │   3086 │   2081 │   1337 │    345 │       6849 │
└───────────────────────┴────────┴────────┴────────┴────────┴────────────┘
\begin{tabular}{rrrrrr}
 

To save the allocation, we first need to convert the institution names to institution IDs. The type allocation code ensures that every institution name in the allocation is unique, and every key in the institution mapping dictionary is unique by definition, so the institution mapping is reversible.

In [4]:
function name_to_id(name, institution_mapping)
    ## returns an institution_id
    for (key, value) in institution_mapping
        if value == name
            return key
        end
    end
    return "not found"
end

name_to_id (generic function with 1 method)

Double-check that there are no duplicate institution names:

In [5]:
length(res.institutions)

1501

In [6]:
length(Set(res.institutions))

1501

Save the allocation:

In [7]:
type_dictionary = []
for (i, alloc) in enumerate(res.allocation)
    inst_id = name_to_id(res.institutions[i], res.institution_mapping)
    push!(type_dictionary, Dict("name" => res.institutions[i], "institution_id" => inst_id, "type" => alloc))
end

In [8]:
open(".estimates/oid_to_type.json", "w") do f
    write(f, JSON.json(type_dictionary))
end;

Re-read the allocation as a sanity check:

In [9]:
for tier in 1:NUMBER_OF_TYPES
    println("TYPE $tier")
    hold = []
    for entry in type_dictionary
        if entry["type"] == tier
            push!(hold, string(" ", entry["name"], " (", entry["institution_id"], ")"))
        end
    end
    sort!(hold)
    for entry in hold
        println(entry)
    end
    println("Total $(length(hold)) institutions")
    println()
end

TYPE 1
 Boston University (29)
 Columbia University (22)
 Duke University (17)
 Harvard University (350)
 London School of Economics and Political Science (351)
 Massachusetts Institute of Technology (311)
 New York University (144)
 Northwestern University (18)
 Princeton University (747)
 Stanford University (78)
 University of California Los Angeles (UCLA) (6)
 University of California, Berkeley (781)
 University of Chicago (339)
 University of Maryland (13)
 University of Michigan (4173)
 University of Pennsylvania (137)
 University of Wisconsin, Madison (73)
 Yale University (290)
Total 18 institutions

TYPE 2
 Arizona State University (370)
 Bocconi University (1034)
 Boston College (312)
 Brown University (21)
 Carnegie Mellon University (158)
 Cornell University (361)
 Johns Hopkins University (40)
 Michigan State University (33)
 National University of Singapore (737)
 Ohio State University (465)
 Pennsylvania State University (134)
 Purdue University (74)
 Stockholm School of